# What is regression ?

According to Wikipedia:
>In statistical modeling, regression analysis is a set of statistical processes for estimating the relationships between a dependent variable (often called the outcome or response variable, or a label in machine learning parlance) and one or more error-free independent variables (often called regressors, predictors, covariates, explanatory variables or features).

In machine learning regression refers to a supervised learning technique where the goal is to predict a continuous numerical value based on one or more independent features.

# What is linear regression ?

According to Wikipedia:
>In statistics, linear regression is a model that estimates the relationship between a scalar response (dependent variable) and one or more explanatory variables (regressor or independent variable) using linear predictor functions whose unknown model parameters are estimated from the data.

In machine learning linear regression is a specific type of regression model that predicts the output as a linear function of the input features.

More specifically, in linear regression, given an input features vector $x=\begin{pmatrix} x_{0}=1 \\ x_{1} \\ x_{2} \\ ... \\ x_{n} \end{pmatrix}$, the model predicts the output according to the following hypothesis function:


$$
h_\Theta(x) = \Theta_{0}\underbrace{x_{0}}_{=1} + \Theta_{1}x_{1} + \Theta_{2}x_{2} + ... + \Theta_{n}x_{n}
$$

$$
h_\Theta(x) = \Theta^{T}x
$$
  
<br><br>
$\Theta_{0}$ is called the bias of the model. It allows the model to shift the regression line predicted up or down so it can better fit the data. 

Without it, the output for $x=\begin{pmatrix} 0 \\ 0 \\ ... \\ 0 \end{pmatrix}$ would always be 0.
